In [ ]:
import wandb

# Initialize a new run
run = wandb.init(project="huggingface", name="my_run")

# Your custom data
my_custom_data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Log the custom data
wandb.log({"eval/loss": wandb.Table(data=my_custom_data, columns=["x", "y", "z"])})

# Finish the run
run.finish()

In [3]:
import wandb

# Initialize the API
api = wandb.Api()

# Load the old run
run = api.run("henry-hoang/huggingface/o3unf6do")

# # Change the data
# run.config["foo"] = 32

# # Update the run
# run.update()


In [8]:
run.history()

,eval/runtime,train/train_loss,train/train_samples_per_second,eval/loss,_timestamp,train/train_runtime,eval/samples_per_second,train/train_steps_per_second,train/loss,train/total_flos,_step,_runtime,train/epoch,train/global_step,train/learning_rate,eval/steps_per_second
0,NaN,NaN,NaN,NaN,1.698851e+09,NaN,NaN,NaN,2.0742,NaN,0,69.514992,0.00,1,9.852217e-08,NaN
1,NaN,NaN,NaN,NaN,1.698852e+09,NaN,NaN,NaN,2.1284,NaN,1,253.796557,0.01,4,3.940887e-07,NaN
2,NaN,NaN,NaN,NaN,1.698852e+09,NaN,NaN,NaN,2.0902,NaN,2,501.275680,0.02,8,7.881773e-07,NaN
3,NaN,NaN,NaN,NaN,1.698852e+09,NaN,NaN,NaN,2.1168,NaN,3,736.185644,0.03,12,1.182266e-06,NaN
4,NaN,NaN,NaN,NaN,1.698852e+09,NaN,NaN,NaN,2.1224,NaN,4,900.045730,0.04,16,1.576355e-06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,1.698976e+09,NaN,NaN,NaN,1.7430,NaN,508,124508.434593,3.48,2016,2.897529e-09,NaN
496,NaN,NaN,NaN,NaN,1.698976e+09,NaN,NaN,NaN,1.7226,NaN,509,124681.516412,3.49,2020,1.478366e-09,NaN
497,NaN,NaN,NaN,NaN,1.698976e+09,NaN,NaN,NaN,1.7166,NaN,510,124939.871342,3.50,2024,5.322201e-10,NaN
498,NaN,NaN,NaN,NaN,1.698977e+09,NaN,NaN,NaN,1.7129,NaN,511,125198.358560,3.51,2028,5.913604e-11,NaN


In [6]:
run.config.keys()

dict_keys(['bf16', 'fp16', 'fsdp', 'seed', 'tf32', 'debug', 'n_ctx', 'optim', 'top_k', 'top_p', 'n_embd', 'n_head', 'prefix', 'do_eval', 'n_inner', 'n_layer', 'no_cuda', 'use_cpu', 'do_train', 'id2label', 'label2id', 'run_name', 'use_ipex', 'adafactor', 'data_seed', 'deepspeed', 'do_sample', 'hub_token', 'log_level', 'max_steps', 'n_special', 'num_beams', 'ray_scope', 'report_to', 'typical_p', 'use_cache', 'adam_beta1', 'adam_beta2', 'attn_pdrop', 'do_predict', 'embd_pdrop', 'eval_delay', 'eval_steps', 'is_decoder', 'local_rank', 'max_length', 'min_length', 'model_type', 'optim_args', 'output_dir', 'past_index', 'save_steps', 'vocab_size', 'ddp_backend', 'ddp_timeout', 'fsdp_config', 'label_names', 'logging_dir', 'n_positions', 'push_to_hub', 'resid_pdrop', 'return_dict', 'sharded_ddp', 'temperature', 'torch_dtype', 'torchdynamo', 'torchscript', 'adam_epsilon', 'bos_token_id', 'disable_tqdm', 'eos_token_id', 'fp16_backend', 'hub_model_id', 'hub_strategy', 'pad_token_id', 'problem_type'